TODO delete this file after debugging test_data_correctness.py

In [1]:
import unittest
import zarr
from dask.array.utils import assert_eq
import dask.array as da
import yaml
import os
from parameterized import parameterized

from src.dataset import NCAR_Dataset
from IPython.core.debugger import set_trace



config = {}
with open('tests/config.yaml', 'r') as file:
    config = yaml.safe_load(file)
dataset_name = os.environ.get('DATASET', 'NCAR-High-Rate-1')
start_timestep = int(os.environ.get('START_TIMESTEP', 0))
end_timestep = int(os.environ.get('END_TIMESTEP', 2))


# Cannot call class method using Parameterized, so have to add this fn. outside the class
def generate_data_correctness_tests():
    global config, dataset_name, start_timestep, end_timestep

    print("start_timestep: ", start_timestep)
    print("end_timestep: ", end_timestep)

    test_params = []
    dataset_config = config['datasets'][dataset_name]
    write_config = config['write_settings']
    dataset = NCAR_Dataset(
        name=dataset_config['name'],
        location_path=dataset_config['location_path'],
        desired_zarr_chunk_size=write_config['desired_zarr_chunk_length'],
        desired_zarr_array_length=write_config['desired_zarr_array_length'],
        prod_or_backup=write_config['prod_or_backup'],
        start_timestep=dataset_config['start_timestep'],
        end_timestep=dataset_config['end_timestep']
    )

    for timestep in range(start_timestep, end_timestep + 1):
        print("Current timestep: ", timestep)
        lazy_zarr_cubes, range_list = dataset.transform_to_zarr(timestep)
        print("len of lazy_zarr_cubes: ", len(lazy_zarr_cubes))
        destination_paths = dataset.get_zarr_array_destinations(timestep, range_list)
        print("len of destination_paths: ", len(destination_paths))
        for original_data_cube, written_zarr_cube in zip(lazy_zarr_cubes, destination_paths):
            test_params.append((original_data_cube, written_zarr_cube))

    print("Done generating tests. Len of test_params: ", len(test_params))
    return test_params

In [2]:
test_params = generate_data_correctness_tests()

start_timestep:  0
end_timestep:  2
Current timestep:  0
len of lazy_zarr_cubes:  64
len of destination_paths:  64
Current timestep:  1
len of lazy_zarr_cubes:  64
len of destination_paths:  64
Current timestep:  2
len of lazy_zarr_cubes:  64
len of destination_paths:  64
Done generating tests. Len of test_params:  192


In [7]:
test_params[0][0]

<xarray.Dataset>
Dimensions:      (nnz: 512, nny: 512, nnx: 512, extra_dim: 1,
                  velocity component (xyz): 3)
Dimensions without coordinates: nnz, nny, nnx, extra_dim,
                                velocity component (xyz)
Data variables:
    temperature  (nnz, nny, nnx, extra_dim) float32 dask.array<chunksize=(64, 64, 64, 1), meta=np.ndarray>
    pressure     (nnz, nny, nnx, extra_dim) float32 dask.array<chunksize=(64, 64, 64, 1), meta=np.ndarray>
    energy       (nnz, nny, nnx, extra_dim) float32 dask.array<chunksize=(64, 64, 64, 1), meta=np.ndarray>
    velocity     (nnz, nny, nnx, velocity component (xyz)) float32 dask.array<chunksize=(64, 64, 64, 3), meta=np.ndarray>
Attributes:
    Simulation conducted by:  Peter Sullivan, pps@ucar.edu
    Dataset built by:         Edward Patton, patton@ucar.edu
    Affiliation:              NCAR/MMM
    Code:                     NCAR-LES
    Case Description:         GABLS 1, Cooling Rate: 0.25 K/hr
    Created:                  2023-11-03 10:09:45 -0600 UTC

In [8]:
test_params[0][1]

'/home/idies/workspace/turb/data01_01/zarr/sabl2048b_01_prod/sabl2048b01_000.zarr'

In [12]:
def test_zarr_group_data(original_subarray, zarr_group_path):
    '''
    Verify correctness of data contained in each zarr group against
    the original data file (NCAR NetCDF)

    Args:
        original_subarray (xarray.Dataset): (Sub)Array of original data that was written as zarr to zar_group_path
        zarr_group_path (str): Location of the sub-chunked data as a Zarr Group
    '''
    zarr_group = zarr.open_group(zarr_group_path, mode='r')
    print("Comparing original 512^3 with ", zarr_group_path)
    for var in original_subarray.data_vars:
        # set_trace()
        assert_eq(original_subarray[var].data, da.from_zarr(zarr_group[var]))
        if config['general_settings']['verbose']:
            print(var, " OK")

In [13]:
for i in range(10):
    test_zarr_group_data(test_params[i][0], test_params[i][1])

Comparing original 512^3 with  /home/idies/workspace/turb/data01_01/zarr/sabl2048b_01_prod/sabl2048b01_000.zarr
Comparing original 512^3 with  /home/idies/workspace/turb/data02_01/zarr/sabl2048b_02_prod/sabl2048b02_000.zarr


AssertionError: found values in 'a' and 'b' which differ by more than the allowed amount

In [14]:
test_params[2][1]

'/home/idies/workspace/turb/data09_01/zarr/sabl2048b_09_prod/sabl2048b09_000.zarr'

In [15]:
test_params[2][0]

<xarray.Dataset>
Dimensions:      (nnz: 512, nny: 512, nnx: 512, extra_dim: 1,
                  velocity component (xyz): 3)
Dimensions without coordinates: nnz, nny, nnx, extra_dim,
                                velocity component (xyz)
Data variables:
    temperature  (nnz, nny, nnx, extra_dim) float32 dask.array<chunksize=(64, 64, 64, 1), meta=np.ndarray>
    pressure     (nnz, nny, nnx, extra_dim) float32 dask.array<chunksize=(64, 64, 64, 1), meta=np.ndarray>
    energy       (nnz, nny, nnx, extra_dim) float32 dask.array<chunksize=(64, 64, 64, 1), meta=np.ndarray>
    velocity     (nnz, nny, nnx, velocity component (xyz)) float32 dask.array<chunksize=(64, 64, 64, 3), meta=np.ndarray>
Attributes:
    Simulation conducted by:  Peter Sullivan, pps@ucar.edu
    Dataset built by:         Edward Patton, patton@ucar.edu
    Affiliation:              NCAR/MMM
    Code:                     NCAR-LES
    Case Description:         GABLS 1, Cooling Rate: 0.25 K/hr
    Created:                  2023-11-03 10:09:45 -0600 UTC

In [22]:
import numpy as np

In [23]:
np.array(test_params[2][0]['energy'][:10,0,0])

array([[1.e-09],
       [1.e-09],
       [1.e-09],
       [1.e-09],
       [1.e-09],
       [1.e-09],
       [1.e-09],
       [1.e-09],
       [1.e-09],
       [1.e-09]], dtype=float32)

In [24]:
zarr_09 = zarr.open_group(test_params[2][1], mode='r')
zarr_09.info

Name,/
Type,zarr.hierarchy.Group
Read-only,True
Store type,zarr.storage.DirectoryStore
No. members,4
No. arrays,4
No. groups,0
Arrays,"energy, pressure, temperature, velocity"


In [25]:
zarr_09['energy'][:10,0,0]

array([[0.04675426],
       [0.02702598],
       [0.02705679],
       [0.03975639],
       [0.05519444],
       [0.05239934],
       [0.05018754],
       [0.04539921],
       [0.03328591],
       [0.04040463]], dtype=float32)